In [ ]:
import pandas as pd
from math import floor as floor
from datetime import datetime as dt
from datetime import timedelta as td

In [ ]:
from google.colab import files
import io

uploaded = files.upload()
file_name = next(iter(uploaded.keys()))
xls = pd.ExcelFile(io.BytesIO(uploaded[file_name]))
WRITE_PATH = f"answers_{file_name.split('.')[0]}.csv"

Saving dataset_5_478.xlsx to dataset_5_478 (1).xlsx


In [ ]:
def cleanup_function(x):
    if isinstance(x, str):
        return x.strip()
    else:
        return x

sku = pd.read_excel(xls, 'SKU MASTER').applymap(cleanup_function)
sales = pd.read_excel(xls, 'Sales').applymap(cleanup_function)
op_stock = pd.read_excel(xls, 'Opening Stock').applymap(cleanup_function)
stock_trans = pd.read_excel(xls, 'STOCK TRANSFER', header=None).applymap(cleanup_function)

In [ ]:
# Init answers
ans = dict.fromkeys(["Q"+str(i) for i in range(1,6)], 0)

In [ ]:
# Helper Function
def print_info(x):
    print(x.shape)
    print(x.columns)
    return x.head()

In [ ]:
print_info(sku)

(50, 4)
Index(['SKU', 'Description', 'Category', 'Price'], dtype='object')


,SKU,Description,Category,Price
0,F001,Prateek England,Fashion,1440
1,F002,Lucy Phillepe,Fashion,1936
2,F003,Badlands,Fashion,1710
3,F004,Allen Dolly,Fashion,3360
4,F005,Bucchi,Fashion,9500


In [ ]:
print_info(sales)

(6200, 5)
Index(['Date', 'SKU', 'Product Name', 'City', 'Sales'], dtype='object')


,Date,SKU,Product Name,City,Sales
0,2023-01-01,F001,Prateek England,Mumbai,25
1,2023-01-01,F002,Lucy Phillepe,Mumbai,88
2,2023-01-01,F003,Badlands,Mumbai,19
3,2023-01-01,F004,Allen Dolly,Mumbai,49
4,2023-01-01,F005,Bucchi,Mumbai,40


In [ ]:
print_info(op_stock)

(50, 7)
Index(['SKU', 'Description', 'Category', 'MUMBAI', 'Pune', 'Aurangabad',
       'Nashik'],
      dtype='object')


,SKU,Description,Category,MUMBAI,Pune,Aurangabad,Nashik
0,F001,Prateek England,Fashion,68,233,87,61
1,F002,Lucy Phillepe,Fashion,77,161,89,73
2,F003,Badlands,Fashion,82,169,93,67
3,F004,Allen Dolly,Fashion,73,184,86,71
4,F005,Bucchi,Fashion,83,238,84,63


In [ ]:
# Read without whole excel sheet
stock_trans = pd.read_excel(xls, "STOCK TRANSFER", header=None).applymap(cleanup_function)
# Replace NA values with last valid observation
stock_trans.iloc[0, :].ffill(inplace=True)

# Extract city names
city_names = stock_trans.iloc[0, 1:].unique()

# Transpose, correct column names, drop first row (cities), group by city.
stock_trans = stock_trans.transpose()
stock_trans.columns = stock_trans.iloc[0, :].to_list()
stock_trans.drop(0, inplace=True)
stock_trans = stock_trans.groupby(["City"]).agg(list)


def regenerate_columns(x, city):
    x = pd.DataFrame(x.loc[city].to_dict()).T
    x = x.set_axis(x.iloc[0], axis=1).iloc[1:]

    return x

stock_trans_lookup = {
    city: regenerate_columns(stock_trans, city) for city in city_names
}

stock_trans_lookup["Pune"].head()

SKU,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,2023-01-09,2023-01-10,...,2023-01-21,2023-01-22,2023-01-23,2023-01-24,2023-01-25,2023-01-26,2023-01-27,2023-01-28,2023-01-29,2023-01-30
F001,19,24,25,22,18,20,18,16,23,18,...,24,22,22,29,29,32,24,33,28,20
F002,24,25,25,16,25,19,22,26,29,28,...,28,26,29,27,27,27,31,23,33,30
F003,27,25,26,19,19,26,29,26,27,22,...,24,30,30,32,23,30,23,30,34,22
F004,24,26,18,25,24,18,20,23,26,26,...,24,19,27,23,29,29,26,31,29,26
F005,20,22,21,20,25,23,20,21,21,28,...,18,29,34,21,34,28,34,26,25,29


In [ ]:
# 1. Over the 30 days of observed sales, what is the total sale value of SKU B003 ?(INTEGER)

price = sku[sku.SKU == 'B003'].Price.sum()
sold = sales[(sales["SKU"] == "B003")].Sales.sum()
ans["Q1"] = sold*price

ans["Q1"]

927180

In [ ]:
# 2. How much percentage of total sale quantity (Volume) did Household category achieve in the first week?
# (Jan 1 to Jan 7, both days included) (FLOAT - 3 decimal places)

household_skus = sku[sku.Category == "Household"].SKU.to_list()
sales_w1 = sales[(sales["Date"] >= dt(2023, 1, 1)) & (sales["Date"] <= dt(2023, 1, 7))]
ans["Q2"] = round(
    sales_w1[(sales_w1.SKU.isin(household_skus))].Sales.sum() / sales_w1.Sales.sum(), 3
)

ans["Q2"]

0.207

In [ ]:
# 3. What is the maximum sale value by a single SKU in a day across all days? (INTEGER)
# (Sale Value = Sale Qty * Price per Qty)

joint = sales.merge(sku[['SKU', 'Price']], on="SKU")
joint['sale_value'] = joint['Sales'] * joint['Price']

ans["Q3"] = joint.sale_value.max()

ans["Q3"]

3578400

In [ ]:
# 4. What is the least revenue generating category across all days? (STRING)

joint = sales.merge(sku[["SKU", "Category", "Price"]], on="SKU")
joint["sale_value"] = joint["Sales"] * joint["Price"]

ans["Q4"] = joint.groupby(["Category"]).agg({"sale_value": "sum"}).idxmin().iloc[0]
ans["Q4"]

'Books'

In [ ]:
# 5. What is the percentage of sale value in Mumbai across all categories and days? (FLOAT - 3 decimal places)

joint = sales.merge(sku[['SKU', 'Price']], on='SKU')
joint['sale_value'] = joint['Sales'] * joint['Price']

ans["Q5"] = round(joint[joint["City"] == "Mumbai"].sale_value.sum() / joint.sale_value.sum(), 3)
ans["Q5"] # Incorrect

0.335

In [ ]:
output = "\n".join([",".join(str(item) for item in items) for items in ans.items()])

with open(WRITE_PATH, "w+") as f:
    f.write(output)

files.download(WRITE_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>